In [ ]:
## Подготовка данных

In [61]:
import datetime
import os
import geopandas as gpd
import pandas as pd
import numpy as np
local_crs = 4326
example_data_path = "./VRP/exp_data/"

In [46]:
dodo = gpd.read_file(os.path.join(example_data_path, "Данные о доходах додо пиццы.geojson")).to_crs(local_crs)
blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
accessibility_matrix = pd.read_pickle(os.path.join(example_data_path, "matrix_mon_0900.pickle"))
demands = pd.read_csv(os.path.join(example_data_path, "second_scenario.csv"))
demands.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

In [47]:
blocks['centroid'] = blocks['geometry'].centroid
blocks.geometry = blocks['centroid']
blocks.drop(columns=['centroid', 'landuse'], inplace=True)
# Получение 40 случайных строк из исходного GeoDataFrame
blocks_40 = blocks.sample(n=40, random_state=1)  # Используйте другой random_state для другой выборки
# Создание нового GeoDataFrame на основе случайных строк
blocks_40 = gpd.GeoDataFrame(blocks_40, geometry='geometry', crs=blocks_40.crs)
blocks_40.reset_index(drop=True, inplace=True)
blocks_40['vrp_id'] = blocks_40.index
blocks_40.head()

C:\Users\danil\AppData\Local\Temp\ipykernel_14816\2260754542.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks['centroid'] = blocks['geometry'].centroid


,geometry,id,vrp_id
0,POINT (30.23725 59.94681),91,0
1,POINT (30.23616 59.93375),443,1
2,POINT (30.27768 59.94933),776,2
3,POINT (30.23291 59.95444),482,3
4,POINT (30.27220 59.93560),1073,4


In [48]:
# Функция для извлечения координат из геометрии
def extract_coordinates(row):
    return row['geometry'].x, row['geometry'].y

# Применение функции к каждой строке GeoDataFrame
blocks_40['coordinates'] = blocks_40.apply(extract_coordinates, axis=1)

# Преобразование координат в массив numpy
coordinates_array = np.array(blocks_40['coordinates'].tolist())

coordinates_array

array([[30.23725223, 59.94681097],
       [30.23615802, 59.93375286],
       [30.277682  , 59.94933284],
       [30.23291334, 59.95444208],
       [30.27220029, 59.9355989 ],
       [30.2920856 , 59.94028113],
       [30.24740838, 59.95465574],
       [30.2465466 , 59.94967585],
       [30.27443905, 59.93945694],
       [30.28112209, 59.94055325],
       [30.26709624, 59.93848723],
       [30.26132438, 59.93312537],
       [30.2253302 , 59.9326824 ],
       [30.20965571, 59.95089336],
       [30.24014745, 59.95016267],
       [30.27370056, 59.93642756],
       [30.23820194, 59.93056522],
       [30.22145889, 59.93357544],
       [30.26736085, 59.95338766],
       [30.28285029, 59.9481095 ],
       [30.2742119 , 59.93677489],
       [30.24658721, 59.93650089],
       [30.27102015, 59.93064191],
       [30.2679881 , 59.93489392],
       [30.25979894, 59.94808359],
       [30.23831776, 59.94709691],
       [30.26626537, 59.94770951],
       [30.22902667, 59.9585606 ],
       [30.22322408,

In [49]:
# Получение списка id кварталов из GeoDataFrame
quarters_ids = blocks_40['id'].tolist()

# Фильтрация матрицы связанности по кварталам из GeoDataFrame
filtered_matrix_df = accessibility_matrix.loc[quarters_ids, quarters_ids]

# Вывод отфильтрованной матрицы связанности
filtered_matrix_df.head()

id,91,443,776,482,1073,810,556,547,215,1093,...,767,744,1126,1118,680,201,1070,977,303,248
id,,,,,,,,,,,,,,,,,,,,,
91,0.0,6.1,11.8,7.9,12.4,15.9,9.5,7.7,12.6,12.7,...,9.6,14.6,14.8,14.6,12.3,12.4,10.3,9.2,9.8,14.8
443,5.0,0.0,13.0,9.9,10.1,15.7,11.5,9.7,12.4,12.5,...,10.8,13.6,14.7,13.6,10.0,13.6,11.5,3.6,11.0,15.4
776,12.2,15.0,0.0,14.5,7.9,6.0,11.5,9.9,5.5,5.6,...,2.1,7.3,4.9,4.9,7.8,6.2,5.2,15.1,7.4,4.3
482,5.3,10.2,14.5,0.0,16.3,19.8,7.5,5.7,16.5,16.6,...,13.5,18.5,18.7,18.5,16.2,11.4,10.9,13.6,9.8,18.7
1073,11.5,11.2,8.7,15.1,0.0,5.6,13.7,10.5,3.4,3.2,...,7.5,3.5,4.6,3.5,0.1,10.3,8.2,7.8,8.0,5.9


In [50]:
array_matrix = filtered_matrix_df.values#.astype(int)
# array_result = array_result.reshape(-1, 1)
array_matrix

array([[ 0. ,  6.1, 11.8, ...,  9.2,  9.8, 14.8],
       [ 5. ,  0. , 13. , ...,  3.6, 11. , 15.4],
       [12.2, 15. ,  0. , ..., 15.1,  7.4,  4.3],
       ...,
       [ 8.3,  3.4, 14.7, ...,  0. , 12.7, 14.2],
       [ 9.3, 12.1,  6.4, ..., 13.1,  0. , 10.6],
       [14.3, 14.9,  4.2, ..., 13.9,  9.5,  0. ]])

In [51]:
demands

,date,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-01 02:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-01-01 04:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15307,2023-09-30 19:00:00,89,63,42,41,81,31,80,0,66,...,78,81,87,68,76,66,105,73,42,50
15308,2023-09-30 20:00:00,71,64,36,35,74,27,69,0,58,...,64,66,75,59,72,58,83,64,30,44
15309,2023-09-30 21:00:00,60,43,28,31,63,18,52,0,50,...,40,56,73,55,68,52,52,55,14,41
15310,2023-09-30 22:00:00,41,0,20,28,50,0,0,0,39,...,0,44,65,44,61,42,0,46,0,43


In [52]:
import pandas as pd

# Предположим, что ваш DataFrame называется orders_df
df = demands
# Преобразуем столбец 'date' в datetime
df['date'] = pd.to_datetime(df['date'])

# Устанавливаем 'date' в качестве индекса
df.set_index('date', inplace=True)

# Агрегируем данные по дням и суммируем заказы
daily_orders = df.resample('D').sum()

# Выводим новый датафрейм
daily_orders


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,651,0,0,0,512,174,497,597,386,182,...,570,436,809,567,807,544,702,0,0,0
2022-01-02,626,0,0,0,485,168,476,570,372,172,...,552,417,769,543,773,521,679,0,0,0
2022-01-03,336,0,0,0,260,93,258,309,198,94,...,298,229,418,294,418,281,366,0,0,0
2022-01-04,308,0,0,0,240,84,235,279,182,84,...,271,206,379,268,383,258,332,0,0,0
2022-01-05,337,0,0,0,263,91,257,308,200,94,...,297,225,415,292,417,282,364,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,466,293,239,251,407,152,405,0,332,336,...,379,488,504,361,419,319,556,418,200,299
2023-09-27,513,319,265,273,452,168,446,0,366,370,...,418,534,554,396,464,353,612,460,221,331
2023-09-28,563,351,288,302,489,185,491,0,404,405,...,458,588,608,433,504,384,675,503,243,360


Период неделя
Цикл начинает из каждого склада для пиццерии вычитать спрос за час
так же задать другую переменную (20 % от 1000), которая задаёт объём склада и минимум, когда срабатывает условие начала доставки 
объём доставки - это пополнить склад пиццерий полностью

40 точек закидываются в случайные кварталы на Ваське

Попробовать среднюю оценку по всем пиццериям (по всем

передавать в ортулс точки и сколько груза отвезти

In [53]:
warehouse_number = 2000
reserves = df.copy()
reserves[reserves.columns] = 0

In [74]:
first = reserves.loc['2023-01-01':'2023-01-02']
first.loc['2023-01-01 00:00:00'] = warehouse_number
warehouses = first.copy()
# first.loc['2022-01-01 00:00:00', '3'] = 800
for i in range(len(df.columns)):
    for j in first.index:
        if str(j) == '2023-01-01 00:00:00':
            continue
        else:
            warehouses.loc[j, str(i)] = warehouses.loc[j-datetime.timedelta(hours=1), str(i)] - df.loc[j, str(i)]

In [75]:
warehouses

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 01:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 02:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 03:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 04:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 05:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 06:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 07:00:00,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
2023-01-01 08:00:00,1000,1000,994,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000


In [9]:
def get_orders_by_date(date, daily_orders, GNN):
    if GNN == True:
        try:
            orders = daily_orders.loc[date]
            demand = np.array(orders)
            # demand[demand== 0] = 500
            demand[0] = 0
            demand = demand/1000
            float_array = demand.astype(float)
            float_array = np.round(float_array, 3)
            return float_array
        except KeyError:
            return np.array([])  # Если дата не найдена, возвращаем пустой список
    else:
        try:
            orders = daily_orders.loc[date]
            orders[0] = 0
            return orders.tolist()
        except KeyError:
            return []  # Если дата не найдена, возвращаем пустой список
# Пример использования
input_date = '2023-09-30'  # Замените на нужную дату
demand = get_orders_by_date(input_date, daily_orders, GNN = True)
print(f"Заказы на {input_date}: {demand}")

Заказы на 2023-09-30: [0.    0.609 0.5   0.523 0.855 0.318 0.849 0.    0.7   0.702 0.809 0.85
 0.607 0.533 0.59  0.716 0.891 0.904 0.354 1.014 0.932 0.969 0.856 0.998
 1.249 1.221 1.021 0.666 1.093 1.089 0.793 1.018 1.054 0.75  0.88  0.669
 1.163 0.869 0.42  0.626]


## OR-Tools

In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection


In [11]:
"""Capacited Vehicles Routing Problem (CVRP)."""

start_node = '91'
# Задать доступное количество транспортных средств (фиксированное)
num_vehicles = 3

# Для каждой даты решить задачу CVRP
input_date = '2023-08-30'  # Замените на нужную дату
demand = get_orders_by_date(input_date, daily_orders, GNN = False)
num_vehicles = 3
time_matrix = array_matrix
time_matrix = time_matrix.astype(int)
total_time = 0

C:\Users\danil\AppData\Local\Temp\ipykernel_14816\3801294542.py:17: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  orders[0] = 0


In [12]:
time_matrix

array([[ 0,  6, 11, ...,  9,  9, 14],
       [ 5,  0, 12, ...,  3, 10, 15],
       [12, 15,  0, ..., 15,  7,  4],
       ...,
       [ 8,  3, 14, ...,  0, 12, 14],
       [ 9, 12,  6, ..., 13,  0, 10],
       [14, 14,  4, ..., 13,  9,  0]])

In [13]:

def create_data_model():
    """Stores the data for the problem."""
    demand[0] = 0  
    data = {}
    data["distance_matrix"] = time_matrix
    data["demands"] = demand
    data["vehicle_capacities"] = [15000, 15000, 15000, 15000]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )
  
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == "__main__":
    main()

Objective: 126
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  14 Load(224) ->  29 Load(793) ->  6 Load(1165) ->  35 Load(1529) ->  18 Load(1817) ->  13 Load(2074) ->  27 Load(2373) ->  3 Load(2652) ->  25 Load(3202) ->  7 Load(3202) ->  38 Load(3388) ->  24 Load(3918) ->  36 Load(4443) ->  26 Load(4903) ->  30 Load(5255) ->  19 Load(5741) ->  2 Load(5998) ->  39 Load(6311) ->  5 Load(6459) ->  32 Load(6935) ->  33 Load(7273) ->  31 Load(7779) ->  9 Load(8129) ->  8 Load(8426) ->  10 Load(8790) ->  34 Load(9188) ->  20 Load(9617) ->  15 Load(9935) ->  4 Load(10319) ->  22 Load(10765) ->  23 Load(11214) ->  11 Load(11602) ->  21 Load(12015) ->  37 Load(12370) ->  16 Load(12739) ->  1 Load(12991) ->  12 Load(13293) ->  17 

## GNN

In [14]:
import torch
import os
import numpy as np
from torch_geometric.data import Data, DataLoader
from VRP.creat_vrp import reward1,creat_instance
from VRP.VRP_Actor import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [15]:
def cal_cost(routes):
    # Вычисление длины каждого маршрута
    total_length = 0.0
    for route in routes:
        route_length = sum(array_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        route_length += array_matrix[0][route[0]]
        route_length += array_matrix[route[-1]][0]
        total_length += route_length
    return total_length

In [16]:
datas = []
n_nodes = 40
actor = 51
x=0 # выбираем набор данных

node_ = coordinates_array
input_date = '2023-08-20'   # Замените на нужную дату
demand_ = get_orders_by_date(input_date, daily_orders, GNN = True)
capcity_ = np.array([8.0]) #настройка вместимости транспорта

# node_ = node_.reshape(-1, n_nodes, 2)
node_,demand_=node_.reshape(-1,n_nodes,2),demand_.reshape(-1,n_nodes)

# data_size = node_.shape[0]
np_matrix = array_matrix
np_matrix = np_matrix.astype(int)
np_matrix = np_matrix.reshape(-1, 1)

edges = np.zeros((n_nodes, n_nodes, 1))
def c_dist(x1, x2):
    # return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** 0.5
    return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** 0.5
for i, (x1, y1) in enumerate(node_[x]):
    for j, (x2, y2) in enumerate(node_[x]):
        d = c_dist((x1, y1), (x2, y2))
        edges[i][j][0] = d
edges_ = edges.reshape(-1, 1)

edges_index = []
for i in range(n_nodes):
    for j in range(n_nodes):
        edges_index.append([i, j])
edges_index = torch.LongTensor(edges_index)
edges_index = edges_index.transpose(dim0=0, dim1=1)

agent = Model(3, 128, 1, 16, conv_laysers=4).to(device)
agent.to(device)
folder = 'trained'
filepath = os.path.join(folder, '%s' % actor)
if os.path.exists(filepath):
        path1 = os.path.join(filepath, 'actor.pt')
        agent.load_state_dict(torch.load(path1, device))

datas_ = []
batch_size1 = 10 # sampling batch_size
for y in range(100):
    data = Data(x=torch.from_numpy(node_[x]).float(), 
                edge_index=edges_index,
                edge_attr=torch.from_numpy(edges_).float(),
                demand=torch.tensor(demand_[x]).unsqueeze(-1).float(),
                capcity=torch.tensor(capcity_[0]).unsqueeze(-1).float())
    datas_.append(data)
dl = DataLoader(datas_, batch_size=batch_size1)

min_tour=[]
min_cost=1000
T=2.5#Temperature hyperparameters

for batch in dl:
    costs = []
    with torch.no_grad():
        batch.to(device)
        tour1, _ = agent(batch, n_nodes * 2, False, T)
        for tour in tour1:
            routes_1 = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
            cost = cal_cost(routes_1)
            costs.append(cost)
        id = np.array(costs).argmin()
        m_cost=np.array(costs).min()
        if m_cost<min_cost:
            min_cost=m_cost
            min_tour=tour1[id]

tour=min_tour.unsqueeze(-2)
print('Problem:VRP''%s' % n_nodes,'/ Average distance:', min_cost)
for i, (data, tour) in enumerate(zip(dl, tour)):
    routes = [r[r != 0] for r in np.split(tour.cpu().numpy(), np.where(tour.cpu().numpy() == 0)[0]) if (r != 0).any()]
    print(routes)

print('_______________')

depot = data.x[0].cpu().numpy()
locs = data.x[1:].cpu().numpy()
demands = data.demand.cpu().numpy()
demands=demands[1:]
total = 0
for veh_number, r in enumerate(routes):
    route_demands = demands[r - 1]
    coords = locs[r - 1, :]
    xs, ys = coords.transpose()
    total_route_demand = sum(route_demands)
    total += total_route_demand*1000
    print(f'Маршрут {veh_number}, Груз на маршруте - {total_route_demand*1000}, Количество посещаемых узлов - {len(r)}')
print('Total demands''%s' % total)

c:\Users\danil\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Problem:VRP40 / Average distance: 408.7
[array([17, 37, 30,  4,  2,  9,  6, 34, 24,  1, 22, 12], dtype=int64), array([ 7, 39,  8, 19,  5, 35, 18, 25, 33, 32, 16, 28, 27], dtype=int64), array([31, 14, 21, 11, 20, 26, 15, 23], dtype=int64), array([10, 29,  3, 13, 36, 38], dtype=int64)]
_______________
Маршрут 0, Груз на маршруте - [7970.], Количество посещаемых узлов - 12
Маршрут 1, Груз на маршруте - [7987.], Количество посещаемых узлов - 13
Маршрут 2, Груз на маршруте - [5789.], Количество посещаемых узлов - 8
Маршрут 3, Груз на маршруте - [3960.], Количество посещаемых узлов - 6
Total demands[25706.]
